In [397]:
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
path = "/Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie"
os.chdir(path)
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import bamboolib
import numpy as np

Current working directory: /Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie


In [398]:
zori_metro = pd.read_csv(path + "/data/Zip_Zri_AllHomesPlusMultifamily.csv")
ca11_18 = pd.read_csv('./data/cali_2011_2018.csv', index_col = None)

#output metro zipcodes
ca_top5 = ['Los Angeles-Long Beach-Anaheim', 'San Francisco-Oakland-Hayward','Riverside-San Bernardino-Ontario',\
'Sacramento--Roseville--Arden-Arcade', 'San Diego-Carlsbad']
ca_metro = zori_metro[zori_metro["Metro"].isin(ca_top5)]
ca_metro_zips = ca_metro[["RegionName"]]

In [399]:
#filter to keep metro areas in rent index only
metro11_18 = pd.merge(ca_metro_zips, ca11_18, on = "RegionName", how = "inner")
print(metro11_18.shape)

(6112, 255)


In [400]:
#filter out rows that have more than 10% missing data
perc = 10.0
min_count = int(((100-perc)/100)*metro11_18.shape[1] + 1)
df_cleaned = metro11_18.dropna(axis = 0, thresh = min_count)

#rename columns
df_cleaned = df_cleaned.rename(columns = {"RegionName": "Zipcode"})

#ensure columns are 5 digits long
df_cleaned["Zipcode"] = df_cleaned["Zipcode"].astype(str)
df_cleaned["Zipcode"] = list(map(lambda x: "0" + x if len(x) ==4 else x, df_cleaned["Zipcode"]))

#drop unused columns
df_cleaned = df_cleaned.drop(["RegionID", "SizeRank"], axis = 1)

#changing index type and frequency
df_cleaned.index = pd.to_datetime(df_cleaned["Date"].astype(str), format = "%Y")

In [401]:
g = df_cleaned.groupby("Zipcode")
df_filled0 = (g.ffill()+g.bfill())/2

In [402]:
#merge back zipcode to fill na for first and last year observation of each zipcode
df_filled0["Zipcode"] = df_cleaned["Zipcode"]
df_filled = df_filled0.groupby("Zipcode").ffill().bfill()

#fill na for zipcode with no data for a variable
df_filled = df_filled.fillna(df_filled.mean(), axis = 0)

In [403]:

df_filled["Zipcode"] = df_cleaned["Zipcode"]

In [424]:
df_filled.to_csv("/Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie/data/metro_CA_2011_2018.csv")


In [423]:
features = df_filled.drop(["avg_rent"], axis = 1)
target = df_filled[["Date", "avg_rent"]]

feature_train =  features[features["Date"] != 2018]
features_test = features[features["Date"] == 2018]

target_train = target[target["Date"] != 2018].drop(["Date"], axis = 1)
target_test = target[target["Date"] == 2018].drop(["Date"], axis = 1)

feature_train.to_csv("/Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie/data/feature_train.csv")
feature_test.to_csv("/Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie/data/feature_test.csv")
target_train.to_csv("/Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie/data/target_train.csv")
target_test.to_csv("/Users/chereowang27/NYCDSA/Capstone/NYCDSA_Capstone/Cherie/data/target_test.csv")